# sql-murder-mystery

# 🔍 SQL Murder Mystery

**Tarih:** 15 Ocak 2018  
**Yer:** SQL City  
**Olay:** Cinayet!

Dedektif, olayla ilgili suç mahalli raporunu sana vermişti... ama onu kaybettin!  
Tek bildiğin, olayın 15 Ocak 2018'de SQL City'de işlendiği.  
Cinayeti çözmek için veritabanındaki tüm ipuçlarını SQL komutlarıyla keşfetmelisin.

🎯 İlk adım: Suç mahalli raporunu bul!

---

💡 İpucu: `crime_scene_report` tablosu sana başlangıçta yardımcı olabilir.


In [1]:
!pip install ipython-sql


   ---------------------------------------- 0.0/44.4 kB ? eta -:--:--
   ---------------------------------------- 44.4/44.4 kB 1.1 MB/s eta 0:00:00


In [2]:
%load_ext sql


In [3]:
import numpy as np 
import pandas as pd

In [5]:
%load_ext sql
%sql sqlite:///sql-murder-mystery.db



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%config SqlMagic.autopandas = True


In [10]:
import prettytable
from sql.run import ResultSet

# DEFAULT stilini güvenli bir stile yönlendir
prettytable.DEFAULT = prettytable.PLAIN_COLUMNS


C:\Users\Hülya\AppData\Local\Temp\ipykernel_12704\2454050519.py:5: DeprecationWarning: the 'PLAIN_COLUMNS' constant is deprecated, use the 'TableStyle' enum instead
  prettytable.DEFAULT = prettytable.PLAIN_COLUMNS


In [11]:
%config SqlMagic.autopandas = True


In [12]:
%%sql
SELECT * FROM crime_scene_report
WHERE date = 20180115 AND city = 'SQL City';


 * sqlite:///sql-murder-mystery.db
Done.


,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


In [13]:
%%sql
SELECT name
FROM sqlite_master
WHERE type='table';


 * sqlite:///sql-murder-mystery.db
Done.


,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [18]:
%%sql
SELECT *
FROM crime_scene_report
LIMIT 10;


 * sqlite:///sql-murder-mystery.db
Done.


,date,type,description,city
0,20180115,robbery,A Man Dressed as Spider-Man Is on a Robbery Spree,NYC
1,20180115,murder,Life? Dont talk to me about life.,Albany
2,20180115,murder,"Mama, I killed a man, put a gun against his he...",Reno
3,20180215,murder,REDACTED REDACTED REDACTED,SQL City
4,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City
5,20180115,theft,Big Bully stole my lunch money!,Chicago
6,20180115,fraud,"Lorem ipsum dolor sit amet, consectetur adipis...",Seattle
7,20170712,theft,"A lone hunter stalks the night, firing arrows ...",SQL City
8,20170820,arson,"Wield the Hammer of Sol with honor, Titan, it ...",SQL City
9,20171110,robbery,The Gjallarhorn shoulder-mounted rocket system...,SQL City


In [19]:
%%sql
SELECT description
FROM crime_scene_report
WHERE date = 20180115
LIMIT 10;


 * sqlite:///sql-murder-mystery.db
Done.


,description
0,A Man Dressed as Spider-Man Is on a Robbery Spree
1,Life? Dont talk to me about life.
2,"Mama, I killed a man, put a gun against his he..."
3,Big Bully stole my lunch money!
4,"Lorem ipsum dolor sit amet, consectetur adipis..."
5,"Hamilton: Lee, do you yield? Burr: You shot hi..."
6,Report Not Found
7,"like a mouse, you know. But do cats eat bats, ..."
8,"Here the other guinea-pig cheered, and was sup..."
9,The Dormouse slowly opened his eyes. ‘I wasn’t...


In [20]:
%%sql
SELECT description
FROM crime_scene_report
WHERE date = 20180115 AND city = 'SQL City' AND type = 'murder';


 * sqlite:///sql-murder-mystery.db
Done.


,description
0,Security footage shows that there were 2 witne...


In [22]:
result = %sql SELECT description FROM crime_scene_report WHERE date = 20180115 AND city = 'SQL City' AND type = 'murder';
print(result["description"][0])


 * sqlite:///sql-murder-mystery.db
Done.
Security footage shows that there were 2 witnesses. The first witness lives at the last house on "Northwestern Dr". The second witness, named Annabel, lives somewhere on "Franklin Ave".


Security footage shows that there were 2 witnesses.

 İlk tanık: Northwestern Dr. üzerindeki son evde yaşıyor
 İkinci tanık: Adı Annabel, Franklin Ave. üzerinde bir yerde yaşıyor

Şimd, bu iki kişiyi veritabanından bulup, interview tablosundan ifadelerini sorgulamak.

In [23]:
%%sql
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;


 * sqlite:///sql-murder-mystery.db
Done.


,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [24]:
%%sql
SELECT *
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave';


 * sqlite:///sql-murder-mystery.db
Done.


,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [25]:
%%sql
SELECT *
FROM interview
WHERE person_id IN (id1, id2);


 * sqlite:///sql-murder-mystery.db
(sqlite3.OperationalError) no such column: id1
[SQL: SELECT *
FROM interview
WHERE person_id IN (id1, id2);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [32]:
result = %sql SELECT * FROM interview WHERE person_id IN (14887, 16371);
result_df = result  

# Tanık ifadelerini ayrı ayrı tam metin olarak yazdıralım
print("Morty Schapiro:\n", result_df[result_df["person_id"] == 14887]["transcript"].values[0])
print("\nAnnabel Miller:\n", result_df[result_df["person_id"] == 16371]["transcript"].values[0])


 * sqlite:///sql-murder-mystery.db
Done.
Morty Schapiro:
 I heard a gunshot and then saw a man run out. He had a "Get Fit Now Gym" bag. The membership number on the bag started with "48Z". Only gold members have those bags. The man got into a car with a plate that included "H42W".

Annabel Miller:
 I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th.


Morty Schapiro:
Bir silah sesi duydum ve ardından bir adamın dışarı kaçtığını gördüm.
Elinde “Get Fit Now Gym” çantası vardı.
Çantadaki üyelik numarası “48Z” ile başlıyordu.
Sadece altın üyeler bu çantalardan alabiliyor.
Adamın bindiği arabada “H42W” içeren bir plaka vardı.

 Annabel Miller:
Cinayetin işlendiğini gördüm ve katili spor salonumdan tanıdım.
Geçen hafta, 9 Ocak’ta spor yaparken onu görmüştüm.

Şüpheli, Get Fit Now Gym üyesi

Altın üyelik seviyesi var

Üyelik numarası "48Z" ile başlıyor

Aracının plakasında "H42W" geçiyor

9 Ocak 2018'de spor salonundaydı

In [33]:
%%sql
SELECT *
FROM get_fit_now_member
WHERE membership_status = 'gold'
  AND id LIKE '48Z%';


 * sqlite:///sql-murder-mystery.db
Done.


,id,person_id,name,membership_start_date,membership_status
0,48Z7A,28819,Joe Germuska,20160305,gold
1,48Z55,67318,Jeremy Bowers,20160101,gold


In [34]:
%%sql
SELECT *
FROM get_fit_now_check_in
WHERE membership_id IN ('48Z7A', '48Z55')
  AND check_in_date = '20180109';


 * sqlite:///sql-murder-mystery.db
Done.


,membership_id,check_in_date,check_in_time,check_out_time
0,48Z7A,20180109,1600,1730
1,48Z55,20180109,1530,1700


In [35]:
%%sql
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%';


 * sqlite:///sql-murder-mystery.db
Done.


,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [36]:
%%sql
SELECT name, license_id
FROM person
WHERE id = 67318;


 * sqlite:///sql-murder-mystery.db
Done.


,name,license_id
0,Jeremy Bowers,423327


In [38]:
%%sql
SELECT name
FROM sqlite_master
WHERE type = 'table';


 * sqlite:///sql-murder-mystery.db
Done.


,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [39]:
%%sql
PRAGMA table_info(solution);


 * sqlite:///sql-murder-mystery.db
Done.


,cid,name,type,notnull,dflt_value,pk
0,0,user,INTEGER,0,None,0
1,1,value,TEXT,0,None,0


In [40]:
%%sql
INSERT INTO solution (user, value)
VALUES (1, 'Jeremy Bowers');


 * sqlite:///sql-murder-mystery.db
1 rows affected.


""


In [42]:
result = %sql SELECT * FROM solution;
print(result['value'][0])



 * sqlite:///sql-murder-mystery.db
Done.
Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.


Jeremy Bowers sadece "görünen katil", gerçek azmettirici biri daha var.

Senden beklenen:

interview tablosunda Jeremy Bowers'ın verdiği ifadeye bakmak

İçinden çıkan ipuçlarıyla asıl suçluyu bulmak

Aynı şekilde solution tablosuna ekleyerek kontrol etmek

In [46]:
result = %sql SELECT transcript FROM interview WHERE person_id = 67318;
print(result['transcript'][0])



 * sqlite:///sql-murder-mystery.db
Done.
I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5" (65") or 5'7" (67"). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.



"Çok parası olan bir kadın tarafından tutuldum.
Adını bilmiyorum ama boyunun yaklaşık 5'5" (165 cm) ya da 5'7" (170 cm) civarında olduğunu biliyorum.
Kızıl saçlı ve Tesla Model S kullanıyor.
2017 Aralık ayında SQL Symphony Konseri'ne 3 kez katıldığını biliyorum."



In [47]:
%%sql
SELECT p.id, p.name, dl.hair_color, dl.height, dl.car_make, dl.car_model, COUNT(*) AS concert_visits
FROM person p
JOIN drivers_license dl ON p.license_id = dl.id
JOIN facebook_event_checkin fb ON p.id = fb.person_id
WHERE dl.gender = 'female'
  AND dl.hair_color = 'red'
  AND dl.height BETWEEN 65 AND 67
  AND dl.car_make = 'Tesla'
  AND dl.car_model = 'Model S'
  AND fb.event_name = 'SQL Symphony Concert'
  AND fb.date BETWEEN '20171201' AND '20171231'
GROUP BY p.id, p.name, dl.hair_color, dl.height, dl.car_make, dl.car_model
HAVING COUNT(*) = 3;


 * sqlite:///sql-murder-mystery.db
Done.


,id,name,hair_color,height,car_make,car_model,concert_visits
0,99716,Miranda Priestly,red,66,Tesla,Model S,3


In [48]:
%%sql
INSERT INTO solution (user, value)
VALUES (2, 'Miranda Priestly');


 * sqlite:///sql-murder-mystery.db
1 rows affected.


""
